Import required libraries:

In [26]:
import requests
import json
import bs4
import os
import sqlite3
import threading

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Set SCRAPE=True in order to download the data from FB

In [27]:
SCRAPE = True

UNIFIED_JSON_PATH = "posts.json"
LOGGING_INTERVAL = 250

In [28]:
TOKEN = "EAACEdEose0cBAIrulgRfAlSs4E9j2pqIuf9MGwNAG6mxke8XvXpH3WEUHsZBKPQOHN8hZBe8iCeVKKNc2XVrkx87Ju4ZBujPm25CJ8wrpNwp5PnZCaHLtN91w0ok2AUZApkI7zZCpCnw9kZB2erv1GoHci50fZBE5wVGmZAZAAZB1qTrX94nTHddKciCW6zTZBZAZB7UIZD"

Define a function to return an empty list if string is not JSON formatted:

In [29]:
def read_json(path):
    try:
        return json.loads(open(path, "r").read())
    except json.decoder.JSONDecodeError:
        return []

def json_append(path, post):
    data = read_json(path)
    data.append(post)
    with open(path, "w") as f:
        f.write(json.dumps(data))

Create a supplemental function to simplify API 

In [30]:
def get(url_):
    return json.loads(requests.get(url_).text)

Define gruop ID variables:

In [31]:
TECHNION_CONFESSIONS_ID = "134517547222780"      # Technion - Haifa
TAU_CONFESSIONS_ID = "561380070875128"           # Tel Aviv University
IDC_CONFESSIONS_ID = "199527394120566"           # Beintchumi - Herzlia
HUJI_CONFESSIONS_ID = "323288791493138"          # Hebrew University - Jerusalem
BGU_CONFESSIONS_ID = "151003595697352"           # Ben Gurion University - Beer Sheva
ARIEL_CONFESSIONS_ID = "465435713853175"         # Ariel University
HAIFA_CONFESSIONS_ID = "417201845375921"         # Haifa University
HADASSAH_CONFESSIONS_ID = "2044836585541512"     # Hadassah
ORT_BRAUDE_CONFESSIONS_ID = "1453869761388065"   # Ort-Braude
BAR_ILAN_CONFESSIONS_ID = "2124618411093048"     # Bar-Ilan
MTA_CONFESSIONS_ID = "157123741522094"           # Tel Aviv-Yafo Academy
YIZRAEL_CONFESSIONS_ID = "1758687200848633"      # Yizrael Valley Academy
IDF_CONFESSIONS_ID = "332027507300337"           # IDF
HOGWARTS_CONFESSIONS_ID = "1685757288142526"     # Hogwarts

Authenticate to Graph API - it is preferred to use manual token from Graph API Explorer because it has better permissions:

In [32]:
# s = "6cc937f2a9dbc9df92600f365c777d1a"
# i = "652869818252649"
# u = "https://graph.facebook.com/oauth/access_token?client_id={id}&client_secret={secret}&grant_type=client_credentials"

# TOKEN = get(u.format(id=i, secret=s))['access_token']

Setup Graph API host and API node:

In [33]:
HOST = "https://graph.facebook.com/"
API_NODE = "v2.12/"
QUERY = "?fields=created_time,message_tags,message,shares,reactions.type(LIKE).limit(0).summary(1).as(like),reactions.type(LOVE).limit(0).summary(1).as(love),reactions.type(HAHA).limit(0).summary(1).as(haha),reactions.type(WOW).limit(0).summary(1).as(wow),reactions.type(SAD).limit(0).summary(1).as(sad),reactions.type(ANGRY).limit(0).summary(1).as(angry)&limit=10"

Define a list of the pages that are going to be scraped in a manner of (id, alias) tuples.

In [34]:
PAGES = [
    (TECHNION_CONFESSIONS_ID, "TECHNION"),
    (TAU_CONFESSIONS_ID, "TAU"),
    (IDC_CONFESSIONS_ID, "IDC"),
    (HUJI_CONFESSIONS_ID, "HUJI"),
    (BGU_CONFESSIONS_ID, "BGU"),
    (ARIEL_CONFESSIONS_ID, "ARIEL"),
    (HAIFA_CONFESSIONS_ID, "HAIFA"),
    (HADASSAH_CONFESSIONS_ID, "HADASSAH"),
    (ORT_BRAUDE_CONFESSIONS_ID, "BRAUDE"),
    (BAR_ILAN_CONFESSIONS_ID, "BIU"),
    (MTA_CONFESSIONS_ID, "MTA"),
    (YIZRAEL_CONFESSIONS_ID, "YIZRAEL"),
    (IDF_CONFESSIONS_ID, "IDF"),
    (HOGWARTS_CONFESSIONS_ID, "HOGWARTS")
]

Container for the scraped posts:

In [35]:
posts = []
last_pages = {}  # This is intended for adding more posts without calling for scraped pages

Create a supplemental builder functions for the API calls' URLs:

In [36]:
def build_url(page_id):
    return HOST + API_NODE + page_id + "/posts" + QUERY + "&access_token={}".format(TOKEN)

def build_comments_url(post_id):
    return HOST + API_NODE + post_id + "/comments" + QUERY + "&access_token={}".format(TOKEN)

Create a function that scrapes a general object (post or comment):

In [37]:
def query_object(raw_object, object_type, object_source, parent_id=None):
    obj = {}   
    
    obj['type'] = object_type
    obj['source'] = object_source
    
    obj['id'] = raw_object['id']
    obj['parent_id'] = parent_id
    
    try:
        obj['message'] = raw_object['message']
    except KeyError:
        obj['message'] = ''

    obj['created_time'] = raw_object['created_time']

    like = raw_object["like"]["summary"]["total_count"]
    love = raw_object["love"]["summary"]["total_count"]
    haha = raw_object["haha"]["summary"]["total_count"]
    wow = raw_object["wow"]["summary"]["total_count"]
    sad = raw_object["sad"]["summary"]["total_count"]
    angry = raw_object["angry"]["summary"]["total_count"]

    obj['like'] = like
    obj['love'] = love
    obj['haha'] = haha
    obj['wow'] = wow
    obj['sad'] = sad
    obj['angry'] = angry
    
    obj['total_reactions'] = sum([like, love, haha, wow, sad, angry])
    
    return obj

Now, create a fumction that will paginate through the posts of a URL and scrape them:

In [38]:
def get_posts(url, source):
    res = get(url)
    if 'data' in res:
        for i, raw_post in enumerate(res['data']):
            post = query_object(raw_post, "POST", source)
            posts.append(post)
            if len(posts) % LOGGING_INTERVAL == 0:
                print("{} is on item #{}".format(source, len(posts)))
    if 'paging' in res:
        if 'next' in res['paging']:
            last_pages[source] = url
            return get_posts(res['paging']['next'], source)

In [39]:
def parse_tag(raw_tag, parent_id, parent_source):
    tag = {}
    
    tag["parent_id"] = parent_id
    tag["source"] = parent_source
    tag["type"] = "TAG"
    
    try:
        tag["tagged_user_id"] = raw_tag["id"]
    except KeyError:
        tag["tagged_user_id"] = None
        
    try:
        tag["tagged_user_name"] = raw_tag["name"]
    except KeyError:
        tag["tagged_user_name"] = None
            
    
    try:
        tag["tagged_user_type"] = raw_tag["type"]
    except KeyError:
        tag["tagged_user_type"] = None
            
    
    return tag

Create a function that will call the API for each post comments, scrape them and get their tags:

In [40]:
def get_post_comments(url, post_id, post_source):
    json_path = "{}.json".format(post_source)
    res = get(url)
    if 'data' in res:
        for raw_comment in res['data']:
            comment = query_object(raw_comment, "COMMENT", post_source, parent_id=post_id)
            posts.append(comment)
            
            if raw_comment.get("message_tags"):
                for raw_tag in raw_comment.get("message_tags"):
                    tag = parse_tag(raw_tag, raw_comment["id"], post_source)
                    posts.append(tag)
            
            if len(posts) % LOGGING_INTERVAL == 0:
                print("{} is on item #{}".format(post_source, len(posts)))
            
    if 'paging' in res:
        if 'next' in res['paging']:
            return get_post_comments(res['paging']['next'], post_id, post_source)

And now, create a function that will scrapes all of the posts' comments and comment tags:

In [41]:
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [42]:
def get_comments(posts_list):
    for post in posts_list:
        get_post_comments(build_comments_url(post["id"]), post["id"], post["source"])

I will use threading to query each Facebook Group in a different thread - this is the worker funciton:

If SCRAPE, the script will run the threads (stop them using >>taskkill /f /im -"python.exe"):

In [43]:
if SCRAPE:
    
    threads = []
    
    for node, name in PAGES:
        t = threading.Thread(target=get_posts, args=(build_url(node), name,))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()
        
else:
    posts = pd.read_csv("posts.csv").to_json(orient='records')

HAIFA is on item #250
BRAUDE is on item #500
HADASSAH is on item #750
HADASSAH is on item #1000
HUJI is on item #1250
ARIEL is on item #1500
TECHNION is on item #1750
IDF is on item #2000
TAU is on item #2250
ARIEL is on item #2500
ARIEL is on item #2750
TAU is on item #3000
HOGWARTS is on item #3250
ARIEL is on item #3500
BGU is on item #3750
HADASSAH is on item #4000
HOGWARTS is on item #4250
IDF is on item #4500
BGU is on item #4750
TAU is on item #5000
BRAUDE is on item #5250
HAIFA is on item #5500
IDC is on item #5750
TAU is on item #6000
HADASSAH is on item #6250
ARIEL is on item #6500
HAIFA is on item #6750
TAU is on item #7000
ARIEL is on item #7250
HUJI is on item #7500
TECHNION is on item #7750
HUJI is on item #8000
ARIEL is on item #8250
ARIEL is on item #8500
BIU is on item #8750
BIU is on item #9000
BIU is on item #9250


In [44]:
with open("posts.json", "w") as f:
    f.write(json.dumps(posts))

In [45]:
if SCRAPE:
    
    threads = []
    
    print("[II] Scraping comments...")
    for posts_list in chunks(posts, 20):
        t = threading.Thread(target=get_comments, args=(posts_list,))
        threads.append(t)
        t.start()

[II] Scraping comments...
IDC is on item #9500
BGU is on item #10000
BGU is on item #10250
TAU is on item #10750
IDC is on item #11250
HUJI is on item #11500
TAU is on item #11750
HADASSAH is on item #12000
IDF is on item #12500
BGU is on item #13500
HADASSAH is on item #13750
ARIEL is on item #14500
ARIEL is on item #14750
TECHNION is on item #15250


Create a dataframe:

In [47]:
df = pd.DataFrame(posts)

Exception in thread Thread-97:
Traceback (most recent call last):
  File "C:\Python3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Python3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-42-22c8534da172>", line 3, in get_comments
    get_post_comments(build_comments_url(post["id"]), post["id"], post["source"])
  File "<ipython-input-40-2469a39ebf38>", line 3, in get_post_comments
    res = get(url)
  File "<ipython-input-30-183a6e4ee947>", line 2, in get
    return json.loads(requests.get(url_).text)
  File "C:\Python3\lib\json\__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "C:\Python3\lib\json\decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Python3\lib\json\decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column

In [ ]:
if not SCRAPE:
    import pandas as pd
    import sqlite3

    df = pd.read_sql("select * from posts", sqlite3.connect("data.db"))

Extract manual post id from the posts:

In [48]:
ID_REGEX = "(#\d+|\d+#){1}"
df["page_id"] = df["message"].str.findall(ID_REGEX).str.get(0)
df["message"].str
df["reply_to"] = df["message"].str.findall(ID_REGEX).str.get(1)
df["message"] = df["message"].str.replace(ID_REGEX, "")
df.head()

,angry,created_time,haha,id,like,love,message,parent_id,sad,source,tagged_user_id,tagged_user_name,tagged_user_type,total_reactions,type,wow,page_id,reply_to
0,0.0,2018-02-22T07:00:00+0000,0.0,134517547222780_148184599189408,0.0,0.0,מאז הסמסטר הראשון מלווה אותי תחושת חוסר ביטחו...,None,1.0,TECHNION,NaN,NaN,NaN,1.0,POST,0.0,#3062,NaN
1,0.0,2018-02-22T06:45:00+0000,0.0,134517547222780_148184395856095,0.0,0.0,תמיד מרגיש לי שהבחור שאיתי נמשך אלי הרבה יותר...,None,0.0,TECHNION,NaN,NaN,NaN,0.0,POST,0.0,#3061,NaN
2,0.0,2018-02-22T06:30:00+0000,0.0,134517547222780_148184269189441,2.0,1.0,"תמיד במהלך היום אני מרגיש סבבה אבל בערב, כשאנ...",None,0.0,TECHNION,NaN,NaN,NaN,3.0,POST,0.0,#3060,NaN
3,0.0,2018-02-22T06:15:00+0000,3.0,134517547222780_148184105856124,7.0,0.0,יש אצלי בחור במעונות שכנראה אין לו מבחנים... ...,None,0.0,TECHNION,NaN,NaN,NaN,10.0,POST,0.0,#3059,NaN
4,0.0,2018-02-22T06:00:00+0000,0.0,134517547222780_148146599193208,6.0,0.0,אני מרגישה שמאז שאני בטכניון הגישה שלי כלפי א...,None,0.0,TECHNION,NaN,NaN,NaN,6.0,POST,0.0,#3058,NaN


Creates a clean message column:

In [49]:
df['clean_message'] = df['message'].str.replace("[^א-ת ]", "")
df['clean_message'] = df['clean_message'].str.replace("[\s]", " ")

Export to SQL:

In [50]:
df.to_sql("posts", sqlite3.connect("data.db"), if_exists="replace")

Exception in thread Thread-175:
Traceback (most recent call last):
  File "C:\Python3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Python3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-42-22c8534da172>", line 3, in get_comments
    get_post_comments(build_comments_url(post["id"]), post["id"], post["source"])
  File "<ipython-input-40-2469a39ebf38>", line 3, in get_post_comments
    res = get(url)
  File "<ipython-input-30-183a6e4ee947>", line 2, in get
    return json.loads(requests.get(url_).text)
  File "C:\Python3\lib\json\__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "C:\Python3\lib\json\decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Python3\lib\json\decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 colum